In [8]:
from openai import OpenAI
import os, json
import requests
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
client = OpenAI(
    base_url = "https://api.together.xyz/v1",
    api_key = os.environ['TOGETHER_API_KEY'],
)
model_name = "meta-llama/Meta-Llama-3-8B-Instruct-Turbo"

In [10]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

In [11]:
messages = [{'role':'user','content':"What's the capital of Italy?"}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The capital of Italy is Rome (Italian: Roma).


In [14]:

def get_embedding(text_input):
    headers = {
        "Authorization": f"Bearer {os.environ["HUGGINGFACE_API_KEY"]}",
        "Content-Type": "application/json"
    }

    # Correct data format for Hugging Face API
    data = {
        "inputs": text_input,  # String input, not a list
        "parameters": {}  # Some models require this field
    }

    response = requests.post(os.environ["EMBEDDING_URL"], headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        return response.json()  # Extract embeddings from JSON response
    else:
        print("Error:", response.text)  # Print error message if request fails
        return None

In [15]:
# Example usage:
user_prompt = "What's new in iPhone 16?"
user_prompt_embeddings = get_embedding(user_prompt)
print(user_prompt_embeddings)  # Check output

[-0.051130492240190506, -0.03487968444824219, 0.06352124363183975, -0.008849618025124073, -0.03721245378255844, -0.04209030792117119, 0.016040321439504623, 0.031201209872961044, -0.012764666229486465, 0.04243600368499756, 0.055488571524620056, 0.013243424706161022, -0.0721069723367691, 0.001392629463225603, 0.09633320569992065, 0.028690965846180916, 0.11511239409446716, -0.14742842316627502, -0.07910007238388062, 0.01615915447473526, -0.036645423620939255, -0.014623756520450115, -0.03976474329829216, -0.02999095432460308, 0.030068710446357727, 0.06143716722726822, 0.0010679634287953377, -0.012805236503481865, -0.01915137842297554, -0.13846465945243835, -0.00038742137257941067, -0.006671341136097908, 0.016682470217347145, 0.026835989207029343, 0.0005176413105800748, -0.038422416895627975, -0.0032086207065731287, 0.034954339265823364, -0.009619117714464664, 0.06379406154155731, 0.029643287882208824, 0.09104535728693008, -0.10143060982227325, -0.024185175076127052, 0.046776752918958664, 0

In [16]:
API_URL = "https://api-inference.huggingface.co/models/sentence-transformers/all-MiniLM-L6-v2"
headers = {"Authorization": f"Bearer {os.environ["HUGGINGFACE_API_KEY"]}"}

In [17]:
def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": {
	"source_sentence": "mouad ayoub",
	"sentences": [
		"mouad ayoub 12",
		"hassan touhami",
		"ggggggggggooooooooood"
	]
},
})


output

[0.7835574150085449, 0.24784895777702332, 0.19391104578971863]

In [18]:
from sentence_transformers import SentenceTransformer
sentences = ["mouad yooo"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings.shape)


(1, 384)


In [19]:
len(embeddings[0])

384

In [20]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""



In [21]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [22]:
data = [iphone_16, samsung_s23]

In [23]:
user_prompt = """What's new in iphone 16?"""

In [42]:
user_prompt_embeddings = model.encode(user_prompt)
user_prompt_embeddings.shape

(384,)

In [ ]:
data_embeddings = model.encode(data)

In [31]:
print(data_embeddings.shape)

(2, 384)


In [44]:
data_similaraty_scores = cosine_similarity(user_prompt_embeddings.reshape(1, -1), data_embeddings)
data_similaraty_scores

array([[0.6264995 , 0.34021363]], dtype=float32)